<a id='top'></a>
## Two Dimensional Models - Diffusive Flow

[Simple Diffusive Flow and Uptake in a Biofilm](#biofilm)

[Simple Advective Flow using a Full Grid Method](#saf_grid)



<hr><a id='biofilm'></a><div style='text-align:right;width=100%'><a href='#top'>top</a></div>

## Simple Diffusive Flow and Uptake in a Biofilm - Explicit Solution

Our first model considers the dynamics of diffusive flow in one spatial dimension. For this example, our model system examines the diffusion of nitrate in a biofilm.  This, for example, is a primary mechanism by which most wastewater treatment systems. In our conceptualization, shown below, the biofilm is attached on one side to a surface which is impervious to diffusion, and exposed to a nitrate-containing wastewater on its outer surface. 

![Biofilm Figure](https://explorer.bee.oregonstate.edu/Topic/Modeling/Images/Biofilm2.png "Biofilm")

For this model, we assume 1) our biofilm varies in only one dimension (through the thickness of the biofilm), and 2) the only processes affecting nitrate distribution throughout the biofilm thickness are 1) diffusion, and 2) biological uptake, captured using Monod saturation kinetics.  We will assume the concentration of bacteria in the biofilm is constant and uniform throughout the biofilm thickness. Our governing equation therefore is:

$\large \frac{dN}{dt} = D \frac{d^2N}{dx^2} - u(N) $ 

where $\large u(N) = \mu_{max} \frac{N}{K_n + N} B \frac{1}{Y_{bn}}  $ is the rate of biological uptake of N at a point in the biofilm.

We will solve this using an **Explicit** procedure using a Full Grid approach, descretizing space and time. We will use this basic grid approach for each of the models below.

![Diffusive Flow Figure](https://explorer.bee.oregonstate.edu/Topic/Modeling/Images/DiffusiveFlowGrid.png "Diffusive Flow Figure") 

We develop an algebraic equivalent of the model PDE by substituting a forward difference for any time-base differentials, and a central difference for the 2nd-order differential term, and solving that algebraic equation for each node using an *explicit* solution procedure. Let ***i*** index a position in space, and ***j*** index a position in time, we can replace the governing differential equation with an algebraic equivalent. Hence:

$\large \frac{dN}{dt} = D \frac{d^2N}{dx^2} - u(N) $

becomes

$\large \frac{\Delta N}{\Delta t} = D \frac{\Delta^2 N}{\Delta x^2} - u(N)  $

$\large \frac{N_{j+1,i}-N_{j,i}}{\Delta t} = D \frac{N_{j,i-1}-2N_{j,i}+N_{j,i+1}}{\Delta x^2} - u(N[j,i]) $

Letting $ \lambda = D \frac{\Delta t}{\Delta x^2} $ and solving for $ N_{j+1,i} $ results in the following:

$ N_{j+1,i} = \lambda N_{j,i-1} + (1 - 2\lambda) N_{j,i} + \lambda N_{j,i+1} - u(N[j,i]) \Delta t$

We will apply this to each **interior** node, starting with the first row, and working our way "up" through time to run the simulation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# geometry
width = 10              # width of biofilm, mm 
dx =  0.5               # length of a compartment/node, m
ncols = int(width/dx)+1 # number of columns (nodes) needed for solution grid

# time
endTime = 2              # length of simulation (hours)
dt = 0.001               # time step (hours)
nrows = int(endTime/dt)  # number of grid rows (nodes) needed for solution grid 

# model parameters
D = 82                   # m2/day
B = 0.0
mumax = 0.055            # maximum growth rate for biotic component (g/g/day)
Kn = 0.2                 # Monod half-velocity parameter (mg/l)
Ybn = 0.15               # yield coefficient, gB produced/gC consumed

print( "dx=", dx, "  dt=", dt, "   lambda=", D*dt/(dx*dx), "   nrows=", nrows, "   ncols=", ncols )

def DiffusionModel():
    # first, make the grid, initialized to zero
    N = np.zeros((nrows,ncols))

    # set left hand boundary to 1 mum/L C.  Note that initial conditions are set to zero already 
    N[:,0] = 1

    # because it is a flux, we will deal with our right hand boundary in our equations below
    # instead of specifiying it here.

    # alternative - zero, except spike throughout day 2
    #day2Start = int(2/dt)
    #day2End = int(3/dt)
    #C[:,0] = 0
    #C[day2Start:day2End,0] = Cin;

    # everything set up, solve the model by looping through cols, then rows, to solve interior nodes

    lamda = D*dt/(dx*dx)

    for j in range(0, nrows-1):   # each row is a time level
        for i in range(1,ncols-1):    # each col is a location in space. We don't set column 0 because it is a boundary condition
            N[j+1,i] = lamda * N[j,i-1] + (1-2*lamda)*N[j,i] + lamda * N[j,i+1] \
                       - ((mumax*N[j,i]/(Kn+N[j,i]))*B/Ybn)*dt

            # to enforce the RHS boundary condition (dC/dx=0), we set the last node to the same value as the node next to it.
            N[j+1,ncols-1] = N[j+1,ncols-2]
        
    return N

# run the model for B=0 (no biological uptake)
B=0
N = DiffusionModel()

# run the model for B = 5
B = 5
NB = DiffusionModel()

# make arrays to hold distances and times for method lines and grid methods.  We'll use these for plotting results
distances = np.arange(0,width+dx/2, dx)    # place the grid node starting on the left-hand boundary
times = np.arange(0,endTime+dt/2, dt)
t = len(times)

# plot results
plt.figure(figsize=(9,5))

plt.subplot(121)
plt.plot(distances,N[0,:], label="T=0")
plt.plot(distances,N[int(t/4),:], label="T=.25")
plt.plot(distances,N[int(t/2),:], label="T=.50")
plt.plot(distances,N[int(3*t/4),:], label="T=.75")
plt.plot(distances,N[-1,:], label="T=1.0")
plt.legend()
plt.grid()
plt.xlabel('Distance')
plt.ylabel('N')
plt.title( 'Diffusion Only (Explicit)')

plt.subplot(122)
plt.plot(distances,NB[0,:], label="T=0")
plt.plot(distances,NB[int(t/4),:], label="T=.25")
plt.plot(distances,NB[int(t/2),:], label="T=.50")
plt.plot(distances,NB[int(3*t/4),:], label="T=.75")
plt.plot(distances,NB[-1,:], label="T=1.0")
plt.legend()
plt.grid()
plt.xlabel('Distance')
plt.ylabel('N')
plt.title( 'Diffusion+Biota (Explicit)')

plt.tight_layout()
plt.show()

### An animation of results

In [ ]:
%matplotlib notebook

from matplotlib import animation

# set up an empty figure
fig = plt.figure()
ax = plt.axes(xlim=(distances[0],distances[-1]), ylim=(0,1.2))
line, = ax.plot([], [], linewidth=2)   # remember the trace, since we will need to update this
plt.xlabel("Distance (m)")
plt.ylabel("Concentration (mg/L)")
plt.grid()
ptext = plt.text(4.6,0.84, "0", fontsize=64, color='lightgray')

# initialization function: set the line to contain no data initially
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    # we need to get the model data for all locations for the "i-th" time slice.
    y = N[i,:]   # replace N with NB to see the version with bacteria included
    line.set_data(distances, y)
    ptext.set_text("{:.3f}".format(i/t))
    return line,

# call the animator.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=t, interval=1)

plt.show()

<hr><a id='aquifer'></a><div style='text-align:right;width=100%'><a href='#top'>top</a></div>

## 2D Steady State Aquifer Flow (Explicit)

In this problem we will examine flow in an aquifer.  Specifically, we will look at a "plan view" (as seen from above) of the aquifer, and model the movement of water across the plan view area in response to differences in water height (head) thoughout the aquifer.  Our basic assumption is that water moves in response to gradients in head according to "diffusion" (second order) kinetics, with the rate of movement controlled by the hydraulic conductivity of the soil.  In this case, 
the governing equation for aquifer flow is:


$\large K_x\frac{\partial ^2h}{\partial x^2} + K_y\frac{\partial ^2h}{\partial y^2} - W = S\frac{\partial h}{\partial t}$

where:
- h = water height (head) compared to some reference point (the top of the "undisturbed" water column in our case) (km)
- K = hydraulic conductivity (km/day),
- S = Specific Storage (vol/vol),
- W = pumping rate (km/day)

We will make some simplifying assumptions:

1.  The soil is isotropic: $K_x = K_y = K$
2.  We are only interested in the steady state solution ($\large \frac {\partial h}{\partial t} = 0$)

Hence, our model reduces to:
$\large \frac{\partial ^2h}{\partial x^2} + \frac{\partial ^2h}{\partial y^2} - \frac{W}{K} = 0$


To solve this, we will develop a grid in x and y, and develop a finite difference replacement for our PDE 


Applying finite differences:

$\Large \frac{\partial ^2h}{\partial x^2}= \frac{h_{j,i+1} - 2h_{j,i} + h_{j,i-1}}{\Delta x^2}$

$\Large \frac{\partial ^2h}{\partial y^2}= \frac{h_{j+1,i} - 2h_{j,i} + h_{j-1,i}}{\Delta y^2}$

We will use a regular grid with $\Delta x  = \Delta y$

In that case our full finite difference replacement is:
$\Large \frac{h_{j,i-1}+h_{j,i+1}+h_{j+1,i}+h_{j-1,i}-4h_{j,i}}{\Delta x^2} - \frac{W}{K} = 0$

If we solve this for $h_{j,i}$, we get:

$\Large  h_{j,i} = \frac {(h_{j,i-1}+h_{j,i+1}+h_{j+1,i}+h_{j-1,i})}{4} - \frac {W \Delta x^2} {(4K)}$

Note that in the equation above, the solution for $ h_{j,i} $ is dependent on its neighbor values, and the neighbor values are dependent on $ h_{j,i} $ we will need to iteratively solve this grid into the solution converges. 

To solve this, we set up a grid representing the x and y dimensions.  Note that if we wanted to solve this as a dynamic problem (retaining dh/dt), we would need to set up a three dimensional grid, with the third dimension used to descretize time.

Because this problem is second-order in both x and y, we will need to specify two boundary conditions in x, and two in y.  In both cases, we will assume that the water height at each outer boundary is 0 


We will visualize the output as a "heat map" showing the water heights across our 2D space.

In [ ]:
import numpy as np
from matplotlib import colors, ticker, cm
import matplotlib.pyplot as plt

# function to convert floating point coords to grid indices - used for visualization only
def CoordToGrid(x, y, width, height, rows, cols):
    x = int((x / width) * cols)
    y = int((y / height) * rows)
    return x, y

# function to solve the grid for one pass, returning an updated grid and a measure of improvement
def SolveGrid(h, dx, wells):
    # first, make a new grid the same dimensions as the starting grid - this is so when we start writing
    # updated grid values, we don't overwrite nodes we need in the next calculations
    (rows, cols) = h.shape
    hNew = np.zeros((rows,cols))
    delta = 0

    for j in range(1, rows - 1):  # iterate through y (note starting, ending index)
        for i in range(1, cols - 1):  # iterate through x (note starting index, ending index)

            # add wells if necessary
            wq = 0   # flow rate from well
            for well in wells:
                if i == well['ix'] and j == well['iy']: # is this well in this grid cell?
                    wq = well['q']

            wq = wq / ( 1000 * dx*dx )     # convert to km/day
            hNew[j,i] = ((h[j-1,i] + h[j+1,i] + h[j,i-1] + h[j,i+1])/4) - (wq*dx*dx/(4*K))
            delta += hNew[j,i] - h[j,i]

    hNew[:,0]  = 0  # apply left hand  boundary condition
    hNew[:,-1] = 0  # apply right hand boundary condition
    hNew[0,:]  = 0  # apply bottom boundary condition
    hNew[-1,:] = 0  # apply top boundary condition

    delta = delta / (rows*cols)   # normalize to delta per grid node
    return hNew, delta

# convenince function to run a full simulation (fully iterate the grid)
def RunModel( h, dx, wells ):
    for well in wells:  # convert well coordinants to grid points
        iWell, jWell = CoordToGrid(well['x'], well['y'], width, height, rows, cols)
        print("Well: x={:.2} ({}), y={:.2} ({}), q={:.3}".format(well['x'], iWell, well['y'], jWell, well['q']))
        well['ix'] = iWell
        well['iy'] = jWell

    delta = 100000  # delta tracks the grid values en route to convergence
    iters = 0
    while np.abs(delta) > 0.00001:
        h, delta = SolveGrid(h, dx, wells)
        iters += 1
    
    # at this point we have solved the grid
    print("Converged after {} iterations".format(iters))
    return h


############################################################
# Main Program
############################################################

width = 2    # units - km
height = 1   # units - km
dx = 0.025   # units - km (25m)
K = 2.2      # units - km/day - reasonably porous aquifer

# get grid dimensions.  Note: we assume that x extends horizontally, y extends vertically on the grid
xdistances = np.arange(0, width + dx / 2, dx)
ydistances = np.arange(0, height + dx / 2, dx)
cols = len(xdistances)
rows = len(ydistances)

print("Grid dimensions: {} rows, {} cols".format(rows, cols))

# make a grid to contain state values through space and time
# and set initial conditions - entire grid is a a fixed value
h = np.full((rows, cols), 0, dtype=np.float)

# define wells to use.  Each well is defined with [X location, Y location, flowrate (assumed constant rate)]
# the flow rate is expressed in m3/day, but in the model this is convered to km3/day
wells = [{'x':0.3, 'y':0.6, 'q':1.0},
         {'x':1.2, 'y':0.3, 'q':100.0} ]

h = RunModel(h, dx, wells)

#  generate heat (countour) map
X, Y = np.meshgrid(xdistances, ydistances)

# Automatic selection of levels works; setting the
# log locator tells contourf to use a log scale:
fig, ax = plt.subplots()
plt.xlabel( 'X position (km)')
plt.ylabel( 'Y position (km)')
fig.set_size_inches(10,4)
cs = ax.contourf(X, Y, h*1000, 30, cmap=cm.PuBu_r)

# report some sample locations
i0, j0 = CoordToGrid( 0.5,0.33, width, height, rows, cols)
i1, j1 = CoordToGrid( 0.5,0.67, width, height, rows, cols)
i2, j2 = CoordToGrid( 1.5,0.33, width, height, rows, cols)
i3, j3 = CoordToGrid( 1.5,0.67, width, height, rows, cols)

plt.text(0.5,0.33," {:.2f}".format(h[j0,i0]*1000))
plt.text(0.5,0.67," {:.2f}".format(h[j1,i1]*1000))
plt.text(1.5,0.33," {:.2f}".format(h[j2,i2]*1000))
plt.text(1.5,0.66," {:.2f}".format(h[j3,i3]*1000))
plt.plot([0.5,0.5,1.5,1.5],[0.33,0.67,0.33,0.67],"+", color='red')


cbar = fig.colorbar(cs)
cbar.set_label( "Water Depression (m)")

plt.show()

## Let's examine a water consumption question with this model

This time, we'll use the model to assess the following situation.  A home is situated on our grid, with a well located at position (0.3,0.6) on our grid, in the same location as our first well above. The house has a well that extends 40m below the existing water table.

Using the following data, we will 1) estimate the existing water table surface from the base elevation for a typical single family dwelling, and 2) examine the effect of placing a water-intensive agricultural operation on the site.

Useful data:

1. Typical water usage in the US is about 100gal/household/day (~0.38m3/hh/day) + 20-80% more for outdoor use. For this problem, assume peak daily water use is 1m3/household/day

2. A water intensive crop uses roughly 0.5 cm/day during peak usage periods (50 m3/ha/day)

Task 1: generate the water surface for the base case (househould use only).  Using this surface, get the water deficit at the location of the house.  How does this compare the the depth of the households well?

Task 2: generate a new water surface by adding a well capable of supporting the peak demands of the crop production system. Assume the footprint of the production systems 0.5 sq km. (Note: 1 sq km = 100ha)

In [ ]:
import numpy as np
from matplotlib import colors, ticker, cm
import matplotlib.pyplot as plt


############################################################
# Main Program
############################################################
# make a grid to contain state values through space and time
# and set initial conditions - entire grid is a a fixed value
h = np.full((rows, cols), 0, dtype=np.float)

# define wells to use.  Each well is defined with {X location, Y location, flowrate (assumed constant rate)}
# the flow rate is expressed in m3/day, but in the model this is convered to km3/day

householdUse = 1.0   # m3/day
cropUse = 50.0 * 100.0 * 0.5    # 100 m3/ha/day * 100 ha/km2 * 0.5km2 = m3/day

# create an array of well dictionaries.  Each array is a dictionary containing info for a partiular well
wells = [{'x':0.3, 'y':0.6, 'q':householdUse}]

# run the model with just the household well
h0 = RunModel(h, dx, wells)

col = wells[0]['ix']
row = wells[0]['iy']
    
waterLevel = h0[row,col]*1000
print( "Scenario 1: Household Pump Water Level={:.3f}{:.3f}".format(waterLevel, h0[row+1,col+1]))


# run with an added well for crop use
wells = ({'x':0.3, 'y':0.6, 'q':householdUse},
         {'x':1.2, 'y':0.3, 'q':cropUse} )

h1 = RunModel(h, dx, wells)

col = wells[0]['ix']
row = wells[0]['iy']
    
waterLevel = h1[row,col]*1000
print( "Scenario 2: Household Pump Water Level (m)={:.3f}{:.3f}".format(waterLevel, h1[row+1,col+1]))


# Automatic selection of levels works; setting the
# log locator tells contourf to use a log scale:
fig, ax = plt.subplots()
plt.xlabel( 'X position (km)')
plt.ylabel( 'Y position (km)')
plt.title("Household Only")
fig.set_size_inches(9,4)
cs = ax.contourf(X, Y, h0*1000, 50, cmap=cm.PuBu_r)

# report some sample locations (same as previous)
plt.text(0.5,0.33,"  {:.2f}".format(h0[j0,i0]*1000))
plt.text(0.5,0.67,"  {:.2f}".format(h0[j1,i1]*1000))
plt.text(1.5,0.33,"  {:.2f}".format(h0[j2,i2]*1000))
plt.text(1.5,0.67,"  {:.2f}".format(h0[j3,i3]*1000))
plt.plot([0.5,0.5,1.5,1.5],[0.33,0.67,0.33,0.67],"+", color='red')

plt.show()

# second scenario
fig, ax = plt.subplots()
plt.xlabel( 'X position (km)')
plt.ylabel( 'Y position (km)')
plt.title("Household + Crop")
fig.set_size_inches(9,4)
cs = ax.contourf(X, Y, h1*1000, 50, cmap=cm.PuBu_r)

plt.text(0.5,0.33,"  {:.2f}".format(h1[j0,i0]*1000))
plt.text(0.5,0.67,"  {:.2f}".format(h1[j1,i1]*1000))
plt.text(1.5,0.33,"  {:.2f}".format(h1[j2,i2]*1000))
plt.text(1.5,0.67,"  {:.2f}".format(h1[j3,i3]*1000))
plt.plot([0.5,0.5,1.5,1.5],[0.33,0.67,0.33,0.67],"+", color='red')

plt.show()
